## Identify Points for StreamStats

__Description__: Tool to identify confluence pair points for tributaries of a specific length, add points to the main stem of a stream network at a specific distance interval, and export a shapefile of the points. For additional details, see the [StreamStats Automation Wiki](https://github.com/Dewberry/usgs-tools/wiki/StreamStats-Automation).

__Input__: Stream grid from the [SteamStats Repository](https://streamstatsags.cr.usgs.gov/StreamGrids/directoryBrowsing.asp), masked using `ClipRaster_withMask.ipynb` and the latitude and longitude of the catchment outlet.

__Output__: A shapefile containing the latitude and longitude of points on the stream grid (confluence and main stem locations).


*Authors*: sputnam@Dewberry.com & slawler@Dewberry.com

### Load libraries and Python options:

In [1]:
import os
import sys
sys.path.append('scripts')
from StreamStats_Points import*
import ID_Confluences
import Calc_Distance

### Load the clipped stream grid, specify the catchment outlet, and the tributary exclusion distance:

##### Specify:

In [2]:
path=r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results' #Load the stream grid raster which was masked by the catchment polygon
name='Rock_Creek.tif'

lat=1925315.186 #[m]; latitude of the catchment outlet
lon=1616784.964 #[m]; longitude of the catchment outlet 

disexl=(5280/2.0)*(0.3048) #Tributary exclusion distance

##### Load the stream grid:

In [3]:
sg = StreamGrid(os.path.join(path, name)) #Open the stream grid raster and create an object

crs=sg.crs_value() #Extract the coordinate reference system value (epsg) for the raster
print("epsg:",crs) 

df = sg.dataframe() #Create a dataframe from the stream grid data
df.replace(255, 0, inplace=True) #Replace 255 with 0, where 255 corresponds to the non-stream cells
df.head(n=2) 

epsg: 5070


,0,1,2,3,4,5,6,7,8,9,...,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Convert the catchment outlet's lat/lon to row/column in the stream grid dataframe and extract the cell size:

In [4]:
pix_x, pix_y =coord2index(sg, lat, lon) #Transform the lat and lon values to the row/column location with the stream grid dataframe
pourpoint=[(pix_x, pix_y)] #Add these values to a list as a touple
print("Pourpoint XY:", pourpoint)

cellsize=sg.cell_size() #Raster cell size in meters
print("The Cell Size:", cellsize)

Pourpoint XY: [(877, 1848)]
The Cell Size: 10.0


### Move up the stream and identify the confluences:

In [5]:
save_confluence, confluence_pairs_orig, nogo=ID_Confluences.main(pourpoint, df)

print("Number of points identified:", len(save_confluence))

Number of points identified: 614


### Remove superflous confluences:

In [6]:
true_confluence, false_confluence=Remove_False_Confluence(save_confluence) #A false confluence is where there are not two points with the same confluence number

print("Number of true confluence points:", len(true_confluence))

Number of true confluence points: 475


##### Identify the original superflous confluence location:

In [7]:
false_points=ID_False_ConfluenceLocs(false_confluence, nogo)

##### Remove any original confluences associated with the false confluences:

In [8]:
confluence_pairs_orig=Remove_False_From_Orig(false_confluence, confluence_pairs_orig)

### Calculate the tributary length:

In [9]:
tributary, mainstem, nogoabs=Calc_Distance.main(df, cellsize, true_confluence, false_points, confluence_pairs_orig)

print((len(mainstem)+len(tributary)), len(mainstem))

475 243


##### Remove confluences with tributaries less than specific length:

In [10]:
incl_tribs=Exclude_Tribs(tributary, disexl)

print(len(incl_tribs))        

18


##### Add labels to differentiate between the mainstem and the tributaries and combine:

In [11]:
mainstem_withlabel=[]
tributary_withlabel=[]

for cell in mainstem:
    mainstem_withlabel.append(cell+('M',))
    
for cell in incl_tribs:
    tributary_withlabel.append(cell+('T',))
    
export_confluences=mainstem_withlabel+tributary_withlabel    

### Save the results:

##### Extract the confluence number, distance, and type of confluence:

In [12]:
export_cnum=[]
export_dis=[]
export_type=[]

for cell in export_confluences:
    export_cnum.append(cell[2])
    export_dis.append(cell[3])    
    export_type.append(cell[4])

##### Transform and save as a shapefile:

In [13]:
longitude, latitude=index2coord(sg, export_confluences)  #Transform the row/column value from the stream grid dataframe to latitude/longitude for each confluence

gdf=geodataframe(longitude, latitude, crs, export_cnum, export_dis, export_type) #Store the longitude/latitude for each confluence in a geodataframe

gdf.to_file(filename = os.path.join(path,'Confluences.shp')) #Export the geodataframe as a shapefule

C:\Users\sputnam\AppData\Local\Continuum\anaconda3\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


# End